In [15]:
# !/usr/bin/python
# coding:utf-8
 
    #load data
def loadSimpDat():
    simpDat = [['I1','I2','I5'],
               ['I1','I2'],
               ['I2','I4'],
               ['I1','I2','I4'],
               ['I1','I3'],
               ['I1','I2','I3','I5'],
               ['I1','I2','I3'],
               ['I2','I5'],
               ['I2','I3','I4'],
               ['I3','I4']]
    return simpDat
 
 # 数据格式化
# 函数的功能是返回指定键的值，如果键不在字典中，将会添加键并将值设置为一个指定值，默认为None
def createInitSet(dataSet):
    #产生初始数据集合
    retDict = {}#将数据集转换成字典形式
    for trans in dataSet:#对于每行数据
        fset = frozenset(trans)
        retDict.setdefault(fset, 0)
        retDict[fset] += 1
    return retDict
 

class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
 
    def inc(self, numOccur):
        self.count += numOccur
 
    def disp(self, ind=1):
        print('   ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)
 
  #创建FP树
def createTree(dataSet, minSup=2):
   #dataSet较为特殊，它是一个字典形式，存放项集和对应的出现次数，在一开始出现次数都为1
    #minSup为最小支持度
    headerTable={}  #存放头指针表
    #第一次遍历数据集， 记录每个数据项的支持度
    for trans in dataSet:#由于dataSet是字典，这个循环是对于字典中的每一个键值
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + 1
 
    #根据最小支持度过滤
    lessThanMinsup = list(filter(lambda k:headerTable[k] < minSup, headerTable.keys()))
    for k in lessThanMinsup:#元素去重
        del(headerTable[k])
 
    freqItemSet = set(headerTable.keys())#去重后的元素
    #如果所有数据都不满足最小支持度，返回None, None
    if len(freqItemSet) == 0:
        return None, None
 
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
 
    retTree = treeNode('φ', 1, None)#构建根节点
    #第二次遍历数据集，构建fp-tree
    for tranSet, count in dataSet.items():
        #根据最小支持度处理一条训练样本，key:样本中的一个样例，value:该样例的的全局支持度
        localD = {}#存放频繁项集元素
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]#加入队列
 
        if len(localD) > 0:#该条数据中中存在频繁项集
            #根据全局频繁项对每个事务中的数据进行排序,等价于 order by p[1] desc, p[0] desc
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: (p[1], p[0]), reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#更新FP树
    return retTree, headerTable
 
 # FP树的生长函数
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  # 检查该元素是否已经存在fp树中
        inTree.children[items[0]].inc(count)  # 计数+1
    else:  # 不存在则添加到fp树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:  # 更新头表
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
 
    if len(items) > 1:  # 截取已排序list的剩余部分，并以当前节点作为父节点
        updateTree(items[1:], inTree.children[items[0]], headerTable, count)
 
 #更新头指针表，整个函数的作用就是为了使树节点中d节点连接可以把相同元素全部连接上
def updateHeader(nodeToTest, targetNode):
    while (nodeToTest.nodeLink != None):  # 找到尾节点
        nodeToTest = nodeToTest.nodeLink#循环到队尾
    nodeToTest.nodeLink = targetNode#将该节点加入
 
 #抽取条件模式基
def ascendTree(leafNode, prefixPath):#prefixPaht用于存放路径，它是一个数组
    """获取当前节点的所有祖先"""
    if leafNode.parent != None:#存在父节点
        prefixPath.append(leafNode.name)#将该节点的名字加入
        ascendTree(leafNode.parent, prefixPath)#递归回溯
 
 
def findPrefixPath(basePat, headTable):#该参数为头指针表指向的该元素的第一个实例
    """获取当前频繁项的所有前缀路径（条件模式基）"""
    condPats = {}#存放前缀路径和计数值
    treeNode = headTable[basePat][1]
    while treeNode != None:#如果当前节点不为空
        # 获取当前频繁项的所有前缀路径（条件模式基）
        prefixPath = []#当前节点作为叶节点
        ascendTree(treeNode, prefixPath)#向上回溯整棵树并把路径存入
        if len(prefixPath) > 1:#因为存储的路径包含该元素本身，要用前缀需要去掉这个元素
            condPats[frozenset(prefixPath[1:])] = treeNode.count # 该条件模式基获得该节点所具有的置信度
        treeNode = treeNode.nodeLink#找到下一个相同元素
    return condPats
 
 #创建条件fp树
def mineTree(inTree, headerTable, minSup=2, preFix=set([]), freqItemList=[]):
    ##排序minSup asc, value asc
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: (p[1][0], p[0]))]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # 通过条件模式基找到的频繁项集
        condPattBases = findPrefixPath(basePat, headerTable)
        # 创建条件fp树
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('condPattBases: ', basePat, condPattBases)
            myCondTree.disp()
            print('*' * 30)
 
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)
 
 
simpDat = loadSimpDat()#读取数据
dictDat = createInitSet(simpDat)#格式化数据
myFPTree, myheader = createTree(dictDat, 3)#创建FP树
myFPTree.disp()
print('*' * 30)
# 获取条件模式基
for key in [v[0] for v in sorted(myheader.items(), key=lambda p: (p[1][0], p[0]), reverse=True)]:
    condPats = findPrefixPath(key, myheader)
    print(key, condPats)
print('*' * 30)
# 创建条件fp树
mineTree(myFPTree, myheader, 2)

    φ   1
       I2   8
          I1   5
             I5   1
             I4   1
             I3   2
                I5   1
          I4   1
          I5   1
          I3   1
             I4   1
       I1   1
          I3   1
       I3   1
          I4   1
******************************
I2 {}
I1 {frozenset({'I2'}): 5}
I3 {frozenset({'I1'}): 1, frozenset({'I2', 'I1'}): 2, frozenset({'I2'}): 1}
I4 {frozenset({'I2'}): 1, frozenset({'I2', 'I1'}): 1, frozenset({'I3', 'I2'}): 1, frozenset({'I3'}): 1}
I5 {frozenset({'I2', 'I1'}): 1, frozenset({'I2', 'I3', 'I1'}): 1, frozenset({'I2'}): 1}
******************************
condPattBases:  I5 {frozenset({'I2', 'I1'}): 1, frozenset({'I2', 'I3', 'I1'}): 1, frozenset({'I2'}): 1}
    φ   1
       I2   3
          I1   2
******************************
condPattBases:  I4 {frozenset({'I2'}): 1, frozenset({'I2', 'I1'}): 1, frozenset({'I3', 'I2'}): 1, frozenset({'I3'}): 1}
    φ   1
       I2   3
          I3   1
       I3   1
******************************

In [ ]:
def main():
    simpDat = loadSimpDat()#读取数据
    dictDat = createInitSet(simpDat)#格式化数据
    myFPTree, myheader = createTree(dictDat, 3)#创建FP树
    myFPTree.disp()
    print('*' * 30)
    # 获取条件模式基
    for key in [v[0] for v in sorted(myheader.items(), key=lambda p: (p[1][0], p[0]), reverse=True)]:
        condPats = findPrefixPath(key, myheader)
        print(key, condPats)
    print('*' * 30)
    # 创建条件fp树
    mineTree(myFPTree, myheader, 2)
    
if __name__ == '__main__':
    main()
    